## Capstone Project Notebook

This note book is an assignement for the Coursea Data Science Course.

In [5]:
import pandas as pd
import numpy as np

In [6]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
